# Plot sea water and air temperature at Buoy "Bahia de Palma"

The temperature time series is represented for selected months and years, the months are overlaid in order to make easier the comparison.

In [1]:
%matplotlib inline
import glob
import os
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import colors
from matplotlib import dates
import datetime, time, calendar
import matplotlib as mpl
import locale
import matplotlib.font_manager as fm
import matplotlib.image as image
locale.setlocale(locale.LC_ALL, 'en_US.utf8')
mpl.rcParams.update({'font.size': 20})
prop = fm.FontProperties(fname='/home/ctroupin/.fonts/Cube-Regular2.ttf')

In [72]:
figdir = "/home/ctroupin/Pictures/SOCIB"
timenow = datetime.datetime.now().strftime('%Y%m%d_%H%M')
figname = "temp_bahiadepalma_{0}".format(timenow) 

In [37]:
file_basename = "http://thredds.socib.es/thredds/dodsC/mooring/conductivity_and_temperature_recorder/"
file_list = ['buoy_bahiadepalma-scb_sbe37005/L1/2014/dep0002_buoy-bahiadepalma_scb-sbe37005_L1_2014-06.nc',
             'buoy_bahiadepalma-scb_sbe37005/L1/2014/dep0002_buoy-bahiadepalma_scb-sbe37005_L1_2014-07.nc',
             'buoy_bahiadepalma-scb_sbe37005/L1/2014/dep0002_buoy-bahiadepalma_scb-sbe37005_L1_2014-08.nc',
             'buoy_bahiadepalma-scb_sbe37005/L1/2015/dep0002_buoy-bahiadepalma_scb-sbe37005_L1_2015-06.nc',
             'buoy_bahiadepalma-scb_sbe37005/L1/2015/dep0002_buoy-bahiadepalma_scb-sbe37005_L1_2015-07.nc',
             'buoy_bahiadepalma-scb_sbe37005/L1/2015/dep0002_buoy-bahiadepalma_scb-sbe37005_L1_2015-08.nc',
             'buoy_bahiadepalma-scb_sbe37007/L1/2016/dep0001_buoy-bahiadepalma_scb-sbe37007_L1_2016-06.nc',
             'buoy_bahiadepalma-scb_sbe37007/L1/2016/dep0001_buoy-bahiadepalma_scb-sbe37007_L1_2016-07.nc',
             'buoy_bahiadepalma-scb_sbe37007/L1/2016/dep0001_buoy-bahiadepalma_scb-sbe37007_L1_2016-08.nc']
file_list = [file_basename + s for s in file_list]

In [38]:
file_basename2 = "http://thredds.socib.es/thredds/dodsC/mooring/weather_station/"
file_list2 = ['buoy_bahiadepalma-scb_met008/L1/2014/dep0002_buoy-bahiadepalma_scb-met008_L1_2014-06.nc',
              'buoy_bahiadepalma-scb_met008/L1/2014/dep0002_buoy-bahiadepalma_scb-met008_L1_2014-07.nc',
              'buoy_bahiadepalma-scb_met008/L1/2014/dep0002_buoy-bahiadepalma_scb-met008_L1_2014-08.nc',
              'buoy_bahiadepalma-scb_met008/L1/2015/dep0002_buoy-bahiadepalma_scb-met008_L1_2015-06.nc',
              'buoy_bahiadepalma-scb_met008/L1/2015/dep0002_buoy-bahiadepalma_scb-met008_L1_2015-07.nc',
              'buoy_bahiadepalma-scb_met008/L1/2015/dep0002_buoy-bahiadepalma_scb-met008_L1_2015-08.nc',
              'buoy_bahiadepalma-scb_met008/L1/2016/dep0002_buoy-bahiadepalma_scb-met008_L1_2016-06.nc',
              'buoy_bahiadepalma-scb_met008/L1/2016/dep0002_buoy-bahiadepalma_scb-met008_L1_2016-07.nc',
              'buoy_bahiadepalma-scb_met008/L1/2016/dep0002_buoy-bahiadepalma_scb-met008_L1_2016-08.nc']
file_list2 = [file_basename2 + s for s in file_list2]

# Load data

In [ ]:
def read_variable_time_list(filelist, variable_name):
    '''
    Read the variable and time from a list
    INPUTS: variable standard name and file list
    OUTPUTS: variable, time, date and year
    '''
    variable = np.array([])
    variableQC = np.array([])
    vartime = np.array([])
    vardate = np.array([])
    varyear = np.array([])
    
    for datafiles in filelist:
        # print(datafiles)
        with netCDF4.Dataset(datafiles) as nc:
            vname = nc.get_variables_by_attributes(standard_name=variable_name)[0]
            varname = vname.name
            varnameQC = vname.ancillary_variables
            varnameQC = np.hstack((varnameQC, nc.variables[varnameQC][:]))
            variable = np.hstack((variable, nc.variables[varname][:]))
            time = nc.variables['time'][:]
            vartime = np.hstack((vartime, time))
            timeunits = nc.variables['time'].units
            varyear = np.hstack((varyear, netCDF4.num2date(time[0], timeunits).year * np.ones_like(time)))
    variable = np.ma.masked_where(variableQC !=1, variable)
    vardate = netCDF4.num2date(vartime, timeunits)
    
    return variable, vartime, vardate, varyear

Read data from the mooring 

In [42]:
watertemperature, watertime, waterdate, wateryear = read_variable_time_list(file_list, 'sea_water_temperature')

Read data from weather station

In [43]:
airtemperature, airtime, airdate, airyear = read_variable_time_list(file_list2, 'air_temperature')

Here we modify the dates to simulate that all the time series are measured the same year (offset) to make the comparison easier.

In [44]:
waterdate = np.array([dd.replace(year=int(wateryear.min())) for dd in waterdate])
airdate = np.array([dd.replace(year=int(airyear.min())) for dd in airdate])

# Make the plot

In [45]:
colorlist = ['c', 'y', 'b']

In [46]:
im = image.imread('/home/ctroupin/Presentations/figures4presentations/logo/logo_socib_square.png')

In [75]:
#mpl.rcParams['font.family'] = prop.get_name()
hfmt = dates.DateFormatter('%B')

fig, ax= plt.subplots(num=None, figsize=(15, 8))

NN = 144
i=0
for years in np.unique(wateryear):
    
    indices = np.where(wateryear == years)[0]
    plotwater = plt.plot(waterdate[indices], 
                         watertemperature[indices],
                         marker = 'o',
                         linewidth = 3,
                         markeredgecolor = colorlist[i],
                         color = colorlist[i], ms=1, label=int(years))
    i += 1
    
hl1 = plt.legend(loc=2, prop=prop)
hl1.set_title('Sea water', prop=prop)
for label in hl1.get_lines():
    label.set_linewidth(3)  
axhl = plt.gca().add_artist(hl1)

i = 0
for years in np.unique(wateryear):
    indices = np.where(airyear == years)[0]
    plotair = plt.plot(airdate[indices][NN/2:-NN/2+1], 
                       np.convolve(airtemperature[indices], np.ones(NN)/NN, 'valid'),
                       marker = '.',
                       linewidth = 1,
                       linestyle = '-',
                       markeredgecolor = colorlist[i],
                       color = colorlist[i], ms=0.5)  
    i += 1
    
hl2 = plt.legend(loc=4, prop=prop, title='Air')
hl2.set_title('Air', prop=prop)
for label in hl2.get_lines():
    label.set_linewidth(1)  # the legend line width
    
ax.xaxis.set_major_locator(dates.MonthLocator())
ax.xaxis.set_minor_locator(dates.DayLocator())
ax.xaxis.set_major_formatter(hfmt)
plt.title('Sea water and air temperature ($^{\circ}$C)\n at Bahia de Palma buoy', fontproperties=prop)
#fig.autofmt_xdate()

plt.grid()
newax = fig.add_axes([0.8, 0.9, 0.1, 0.1], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
for label in ax.get_xticklabels():
    label.set_fontproperties(prop)

for label in ax.get_yticklabels():
    label.set_fontproperties(prop)

plt.savefig(os.path.join(figdir, figname), dpi=300)
# plt.show()
plt.close()

In [48]:
figdir, figname

('/home/ctroupin/Pictures/SOCIB', 'temp_bahiadepalma_20160826_1516')